In [1]:
from cobra import Metabolite
from cobra import Reaction
import cameo
import cobra

In [2]:
#importing iJO1366 model
model = cameo.models.bigg.iML1515
model.reactions.EX_co2_e.lower_bound = 0

#metabolite declaration
akg = model.metabolites.get_by_id('akg_c')
coa = model.metabolites.get_by_id('coa_c')
oxo2_coa = Metabolite('oxo2_coa_c', formula = 'C26H34N7O20P3S', name = '2-oxoglutaryl-CoA', compartment = 'c', charge = -6)
h2o = model.metabolites.get_by_id('h2o_c')
H52F_coa = Metabolite('H52F_coa_c', formula = 'C26H32N7O19P3S', name = '5-hydroxy-2-furoyl-CoA', compartment = 'c', charge = -6)
nadh = model.metabolites.get_by_id('nadh_c')
f2_coa = Metabolite('f2_coa_c', formula = 'C26H31N7O18P3S', name = '2-furoyl-CoA', compartment = 'c', charge = -7)
nad = model.metabolites.get_by_id('nad_c')
ppi = model.metabolites.get_by_id('ppi_c')
amp = model.metabolites.get_by_id('amp_c')
H = model.metabolites.get_by_id('h_c')
furoate = Metabolite('fur_c', formula = 'C5H3O3', name = '2-furoate', compartment = 'c', charge = -1)
atp = model.metabolites.get_by_id('atp_c')
co2 = model.metabolites.get_by_id('co2_c')
fur25_ate = Metabolite('fur25_ate_c', formula = 'C6H3O5', name = '2,5-furandicarboxylate', compartment = 'c', charge = -1)
h2o2 = model.metabolites.get_by_id('h2o2_c')
o2 = model.metabolites.get_by_id('o2_c')
HMF = Metabolite('HMF_c', formula = 'C6H6O3', name = '5-Hydroxymethylfurfural', compartment = 'c')


Using license file /Users/sartaajkhan/gurobi.lic
Academic license - for non-commercial use only


In [3]:
#reaction 1: akg + coa <-> oxo2_coa + h2o
reaction1 = Reaction('akg_oxo2_coa')
reaction1.name = '2-oxoglutarate to 2-oxoglutaryl-CoA'
reaction1.lower_bound = -1000
reaction1.upper_bound = 1000
reaction1.add_metabolites({akg: -1.0, coa: -1.0, oxo2_coa: 1.0, h2o: 1.0})
model.add_reaction(reaction1)

#reaction 2: oxo2_coa <-> H52F_coa + h2o
reaction2 = Reaction('oxo2_coa_H52F_coa')
reaction2.name = '2-oxoglutaryl-CoA to 5-hydroxy-2-furoyl-CoA'
reaction2.lower_bound = -1000
reaction2.upper_bound = 1000
reaction2.add_metabolites({oxo2_coa: -1.0, H52F_coa: 1.0, h2o: 1.0})
model.add_reaction(reaction2)

#reaction 3: H52F_coa + nadh <-> f2_coa + nad + h2o
reaction3 = Reaction('H52F_coa_f2_coa')
reaction3.name = '5-hydroxy-2-furoyl-CoA to 2-furoyl-CoA'
reaction3.lower_bound = -1000
reaction3.upper_bound = 1000
reaction3.add_metabolites({H52F_coa: -1.0, nadh: -1.0, f2_coa: 1.0, nad: 1.0, h2o: 1.0})
model.add_reaction(reaction3)

#reaction 4: f2_coa + ppi + amp + 3H <-> furoate + coa + atp
reaction4 = Reaction('f2_coa_furoate')
reaction4.name = '2-furoyl-CoA to 2-furoate'
reaction4.lower_bound = -1000
reaction4.upper_bound = 1000
reaction4.add_metabolites({f2_coa: -1.0, ppi: -1.0, amp: -1.0, H: -3.0, furoate: 1.0, coa: 1.0, atp: 1.0})
model.add_reaction(reaction4)

#reaction 5: furoate + co2 <-> fur25_ate
reaction5 = Reaction('furoate_fur25_ate')
reaction5.name = '2-furoate to 2,5-furandicarboxylate'
reaction5.lower_bound = -1000
reaction5.upper_bound = 1000
reaction5.add_metabolites({furoate: -1.0, co2: -1.0, fur25_ate: 1.0})
model.add_reaction(reaction5)

#reaction 6: fur25_ate + 3h2o2 + H <-> HMF + 3o2 + 2h2o
reaction6 = Reaction('fur25_ate_HMF')
reaction6.name = '2,5-furandicarboxylate to HMF'
reaction6.lower_bound = -1000
reaction6.upper_bound = 1000
reaction6.add_metabolites({fur25_ate: -1.0, h2o2: -3.0, H: -1.0, HMF: 1.0, o2: 3.0, h2o: 2.0})
model.add_reaction(reaction6)

#reaction 7: HMF ->
reaction7 = Reaction('EX_HMF_e')
reaction7.name = 'HMF consumption'
reaction7.lower_bound = 0
reaction7.upper_bound = 1000
reaction7.add_metabolites({HMF: -1.0})
model.add_reaction(reaction7)


In [4]:
#optimum flux calculation
model.objective = 'EX_HMF_e'
max_flux = 0
max_flux=(model.optimize(objective_sense='maximize').objective_value)
print(max_flux)

10.000000000000002


In [5]:
model.summary()

In [6]:
for i in range(-1,-8,-1):
    if model.reactions[i].check_mass_balance() == {}:
        pass
    else:
        print(model.reactions[i])

EX_HMF_e: HMF_c --> 


In [7]:
cobra.io.write_sbml_model(model, "HMF_iML1515.xml")